## Explore the API

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

import pandas as pd


In [ ]:
client_id = "client_id"
client_secret = "client_secret"

In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri='http://127.0.0.1:8888/callback',
    scope='user-top-read user-read-recently-played'
))


In [ ]:
# get top tracks
top_tracks = sp.current_user_top_tracks(limit=10, time_range='short_term')

print("🔥 Your Top Tracks (last 4 weeks):")
for i, track in enumerate(top_tracks['items']):
    name = track['name']
    artist = track['artists'][0]['name']
    print(f"{i+1}: {name} by {artist}")

## Recent Tracks

In [ ]:
# Get recently played tracks (max 50)
recent_tracks = sp.current_user_recently_played(limit=50)

In [ ]:
# Build the data
data = []

for item in recent_tracks['items']:
    track = item['track']
    data.append({
        'track_name': track['name'],
        'artist_name': ', '.join([artist['name'] for artist in track['artists']]),
        'album': track['album']['name'],
        'played_at': item['played_at'],
        'track_id': track['id']
    })

df = pd.DataFrame(data)

## Top Tracks

In [ ]:
top_tracks = sp.current_user_top_tracks(limit=50, time_range='short_term')

# Build DataFrame
data = []

for item in top_tracks['items']:
    track = item['name']
    artist = ', '.join([a['name'] for a in item['artists']])
    album = item['album']['name']
    release_date = item['album']['release_date']
    duration_ms = item['duration_ms']
    popularity = item['popularity']
    track_id = item['id']
    artist_ids = ', '.join([a['id'] for a in item['artists']])
    data.append({
        'track_name': track,
        'artist_name': artist,
        'album_name': album,
        'release_date': release_date,
        'duration_ms': duration_ms,
        'popularity': popularity,
        'track_id': track_id,
        'artist_ids': artist_ids
    })

df = pd.DataFrame(data)

In [ ]:
# additional info
df['rank'] = df.index + 1
df['release_year'] = df['release_date'].apply(lambda x: x.split('-')[0])


In [ ]:
df.to_csv("files/top_tracks.csv", index=False)

In [ ]:
artist_counts = df['artist_name'].value_counts()
print(artist_counts.head(10))


In [ ]:
df['release_year'] = df['release_date'].str[:4]
year_counts = df['release_year'].value_counts().sort_index()


In [ ]:
year_counts

## Audio Features

nah man this was deprecated from the api

check out: 

https://blog.metabrainz.org/2024/11/28/pissed-off-by-spotify-enshittifying-more-api-endpoints-we-can-help/

https://medium.com/data-science/spotify-api-audio-features-5d8bcbd780b2

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp_no_auth = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret
))


In [ ]:
sp_no_auth.audio_features(tracks=['2fuCquhmrzHpu5xcA1ci9x'])

## Genre via Artist

In [ ]:
artist_ids = []

for item in top_tracks['items']:
    for artist in item['artists']:
        artist_ids.append({
            'track_id': item['id'],
            'artist_name': artist['name'],
            'artist_id': artist['id']
        })

artist_df = pd.DataFrame(artist_ids)


In [ ]:
artist_df

In [ ]:
import time

genre_data = []

unique_ids = artist_df['artist_id'].dropna().unique()

for artist_id in unique_ids:
    try:
        artist_info = sp.artist(artist_id)
        genre_data.append({
            'artist_id': artist_id,
            'artist_name': artist_info['name'],
            'followers': artist_info['followers']['total'],
            'popularity': artist_info['popularity'],
            'genres': artist_info['genres']
        })
    except Exception as e:
        print(f"Error fetching genres for {artist_id}: {e}")
    time.sleep(0.1)  # be kind to the API

genres_df = pd.DataFrame(genre_data)


In [ ]:
genres_df

In [ ]:
from collections import Counter

# Flatten the genre lists
all_genres = [genre for sublist in genres_df['genres'].dropna() for genre in sublist]
genre_counts = Counter(all_genres)

# Top 10 genres
print(genre_counts.most_common(10))


In [ ]:
import matplotlib.pyplot as plt

top_genres = genre_counts.most_common(10)
genres, counts = zip(*top_genres)

plt.barh(genres[::-1], counts[::-1])
plt.title("Top Genres in Your Short-Term Top 50")
plt.xlabel("Count")
plt.tight_layout()
plt.show()
